In [1]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/labeled_training (2).csv')
df_dev = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task6_Dev.csv')

In [2]:
import re
import pandas as pd
def preprocess_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = text.lower()
    text = re.sub(r'[@#]\w+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
df_train['text'] = df_train['text'].apply(preprocess_text)
df_dev['text'] = df_dev['text'].apply(preprocess_text)

In [3]:
!pip install transformers seqeval torch tqdm accelerate sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 804.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 M

In [4]:
!pip install accelerate bitsandbytes torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 11.0 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
df_train

,id,text,source,label
0,1,at 50 votes i will close this survey and take ...,twitter,0
1,2,"21st birthday and nearly 4 months pregnant, no...",twitter,1
2,3,// i'd be killed. he's literally my 10th picture.,twitter,0
3,4,"this year i'm gonna become a mum and turn 20, ...",twitter,1
4,5,don't mind me. just freaking out because mary-...,twitter,0
...,...,...,...,...
8795,8796,happy birthday brandon james. hope you have a ...,twitter,0
8796,8797,"at age 50, i remember cheering ""bananas"" befor...",twitter,0
8797,8798,them springfield mall pictures.. lol.,twitter,1
8798,8799,when we turn 18 were going idgaf where its at....,twitter,0


In [8]:
df_dev

,id,text,source,label
0,8801,everybody developa cataracts with age. disease...,reddit,0
1,8802,"i have not heard that before. i have icl's, bu...",reddit,0
2,8803,what does your doctor/fitter say? i sure wish ...,reddit,0
3,8804,dear doctor of optometrists of the internet i ...,reddit,1
4,8805,"if you have sharp vision without correction, p...",reddit,0
...,...,...,...,...
4195,12996,why cant people just accept no as an answer! -...,reddit,1
4196,12997,even the thought of this terrifies me. im 18 a...,reddit,1
4197,12998,i don't think exposure works for me either. i ...,reddit,0
4198,12999,i'm a 16 year old going into junior year of hi...,reddit,1


In [11]:
input_chat = "ONLY Respond with YES or NO ,nothing else before or after the answer. Can the exact age of the AUTHOR (in years) be known from the texts: i was in rgp's 1968-1982. these are so much more comfortable. they also do not touch the cornea. my transplants are 39 and 41 years old. ONLY Respond with YES or NO "
input_ids = tokenizer.encode(input_chat, return_tensors='pt')
output_ids = model.generate(input_ids, max_new_tokens=20, num_return_sequences=5, do_sample=True)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[1], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[2], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[3], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[4], skip_special_tokens=True)
print(output_text)

ONLY Respond with YES or NO ,nothing else before or after the answer. Can the exact age of the AUTHOR (in years) be known from the texts: i was in rgp's 1968-1982. these are so much more comfortable. they also do not touch the cornea. my transplants are 39 and 41 years old. ONLY Respond with YES or NO 
ONLY Respond with YES or NO ,nothing else before or after the answer. Can the exact age of the AUTHOR (in years) be known from the texts: i was in rgp's 1968-1982. these are so much more comfortable. they also do not touch the cornea. my transplants are 39 and 41 years old. ONLY Respond with YES or NO 
ONLY Respond with YES or NO ,nothing else before or after the answer. Can the exact age of the AUTHOR (in years) be known from the texts: i was in rgp's 1968-1982. these are so much more comfortable. they also do not touch the cornea. my transplants are 39 and 41 years old. ONLY Respond with YES or NO 
ONLY Respond with YES or NO ,nothing else before or after the answer. Can the exact age 

In [17]:
input_chat = "Can the age of the AUTHOR in years be known from the texts? (yes/no) : they told me my peripheral retina has thinnings and had laser photocoagulation twice for lattice and horse shoe tear in peripheral. i'm 19. similar rx as yours."
input_ids = tokenizer.encode(input_chat, return_tensors='pt')
output_ids = model.generate(input_ids, max_new_tokens=10, num_return_sequences=5, do_sample=True)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[1], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[2], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[3], skip_special_tokens=True)
print(output_text)
output_text = tokenizer.decode(output_ids[4], skip_special_tokens=True)
print(output_text)

Can the age of the AUTHOR in years be known from the texts? (yes/no) : they told me my peripheral retina has thinnings and had laser photocoagulation twice for lattice and horse shoe tear in peripheral. i'm 19. similar rx as yours.

The text mentions the author's age
Can the age of the AUTHOR in years be known from the texts? (yes/no) : they told me my peripheral retina has thinnings and had laser photocoagulation twice for lattice and horse shoe tear in peripheral. i'm 19. similar rx as yours.

This text is written by a 1
Can the age of the AUTHOR in years be known from the texts? (yes/no) : they told me my peripheral retina has thinnings and had laser photocoagulation twice for lattice and horse shoe tear in peripheral. i'm 19. similar rx as yours.

Answer: No, the age of the
Can the age of the AUTHOR in years be known from the texts? (yes/no) : they told me my peripheral retina has thinnings and had laser photocoagulation twice for lattice and horse shoe tear in peripheral. i'm 19. 